<a href="https://colab.research.google.com/github/TimeIsTheChoice/ColabCaffe/blob/master/brewing_logreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Caffe Prerpare

In [0]:
#@title Check Cuda_Tools_Version {display-mode: "form"}
!nvidia-smi
!nvcc -V
# This code will be hidden when the notebook is loaded.


In [0]:
#@title Check cuda tools version {display-mode: "form"}
If_not_10_0_130_click_box = False #@param {type:"boolean"}
if If_not_10_0_130_click_box:
  !apt  --purge remove "cublas*" "cuda*"
  !apt  --purge remove "nvidia*" 
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
  !apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub && sudo apt update
  !dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
  #!apt update
  !apt install -y cuda=10.0.130-1

  # Install CuDNN 7 and NCCL 2
  !wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
  !dpkg -i nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

  #!apt update
  !apt install -y libcudnn7 libcudnn7-dev libnccl2 libc-ares-dev

  !apt autoremove
  #!apt upgrade

  # Link libraries to standard locations
  !mkdir -p /usr/local/cuda-10.0/nccl/lib
  !ln -s /usr/lib/x86_64-linux-gnu/libnccl.so.2 /usr/local/cuda/nccl/lib/
  !ln -s /usr/lib/x86_64-linux-gnu/libcudnn.so.7 /usr/local/cuda-10.0/lib64/  
else:
  print("version correct,goning on!")
# This code will be hidden when the notebook is loaded.


In [0]:
#@title clone caffe and install dependence {display-mode: "form"}
dirname = os.path.abspath('/content/caffe')
if not os.path.exists(dirname):
  !apt install python3-opencv
  !apt-get install libopenblas-dev # OpenBLAS
  # Other dependencies
  !apt-get install libprotobuf-dev libleveldb-dev libsnappy-dev libopencv-dev libhdf5-serial-dev protobuf-compiler
  !apt-get install — no-install-recommends libboost-all-dev
  !apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
  !pip3 install protobuf
  !apt-get install the python3-dev

  import os
  os.chdir('/content')
  !rm -rf caffe
  !git clone https://github.com/TimeIsTheChoice/caffe.git
  !cd caffe
else:
  print("Caffe exist!")

# This code will be hidden when the notebook is loaded.


In [0]:
#@title Get cpu cores number {display-mode: "form"}
!nproc --all
!echo "Threads/core: $(nproc --all)"
# This code will be hidden when the notebook is loaded.


In [0]:
#@title If build faild,click me! {display-mode: "form"}
Build_Failed = False #@param {type:"boolean"}
if Build_Failed:
  !make clean
else:
  print("nothing happen!")
# This code will be hidden when the notebook is loaded.


In [0]:
#@title Build Caffe {display-mode: "form"}
Build_Caffe = False #@param {type:"boolean"}
if Build_Caffe:
  import os
  os.chdir('/content/caffe')
  !make all -j$(nproc --all) # 4 represents number of CPU Cores
  !make pycaffe -j$(nproc --all) # 4 represents number of CPU Cores
  !export PYTHONPATH=~/caffe/python:$PYTHONPATH
else:
  print("Make sure you want build it,please!")
# This code will be hidden when the notebook is loaded.


In [0]:
#@title check Caffe  {display-mode: "form"}
caffe_root = '/content/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)

import sys
sys.path.insert(0, caffe_root + 'python')
import caffe
print("import successful!")
# This code will be hidden when the notebook is loaded.


In [0]:
#@title Check runtime whetehr gpu or not {display-mode: "form"}
#' ' means CPU whereas '/device:G:0' means GPU
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
# This code will be hidden when the notebook is loaded.


In [0]:
#@title check GPU memory utilization {display-mode: "form"}
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 
# This code will be hidden when the notebook is loaded.


# Brewing Logistic Regression then Going Deeper

While Caffe is made for deep networks it can likewise represent "shallow" models like logistic regression for classification. We'll do simple logistic regression on synthetic data that we'll generate and save to HDF5 to feed vectors to Caffe. Once that model is done, we'll add layers to improve accuracy. That's what Caffe is about: define a model, experiment, and then deploy.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('..')

import sys
sys.path.insert(0, './python')
import caffe


import os
import h5py
import shutil
import tempfile

import sklearn
import sklearn.datasets
import sklearn.linear_model

import pandas as pd

Synthesize a dataset of 10,000 4-vectors for binary classification with 2 informative features and 2 noise features.

In [0]:
X, y = sklearn.datasets.make_classification(
    n_samples=10000, n_features=4, n_redundant=0, n_informative=2, 
    n_clusters_per_class=2, hypercube=False, random_state=0
)

# Split into train and test
X, Xt, y, yt = sklearn.model_selection.train_test_split(X, y)

# Visualize sample of the data
ind = np.random.permutation(X.shape[0])[:1000]
df = pd.DataFrame(X[ind])
_ = pd.plotting.scatter_matrix(df, figsize=(9, 9), diagonal='kde', marker='o', s=40, alpha=.4, c=y[ind])

## Train and Test
>Learn and evaluate scikit-learn's logistic regression with stochastic gradient descent (SGD) training. Time and check the classifier's accuracy.



alter
```
clf = sklearn.linear_model.SGDClassifier(
    loss='log', n_iter =1000, penalty='l2', alpha=5e-4, class_weight='balanced')
```
to


```
clf = sklearn.linear_model.SGDClassifier(
    loss='log', max_iter=1000, penalty='l2', alpha=5e-4, class_weight='balanced')
```
cite [How to fix TypeError: __init__() got an unexpected keyword argument 'max_iter'?](https://forums.databricks.com/questions/20232/how-to-fix-typeerror-init-got-an-unexpected-keywor.html)



In [0]:
%%timeit
# Train and test the scikit-learn SGD logistic regression.
clf = sklearn.linear_model.SGDClassifier(
    loss='log', max_iter=1000, penalty='l2', alpha=5e-4, class_weight='balanced')

clf.fit(X, y)
yt_pred = clf.predict(Xt)
print('Accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(yt, yt_pred)))

## Save the dataset to HDF5 

In [0]:
# Write out the data to HDF5 files in a temp directory.
# This file is assumed to be caffe_root/examples/hdf5_classification.ipynb
dirname = os.path.abspath('/content/caffe/examples/hdf5_classification/data')
if not os.path.exists(dirname):
    os.makedirs(dirname)

train_filename = os.path.join(dirname, 'train.h5')
test_filename = os.path.join(dirname, 'test.h5')

# HDF5DataLayer source should be a file containing a list of HDF5 filenames.
# To show this off, we'll list the same data file twice.

>such as

```
/content/caffe/examples/hdf5_classification/data/train.h5
/content/caffe/examples/hdf5_classification/data/train.h5
```



In [0]:

with h5py.File(train_filename, 'w') as f:
    f['data'] = X
    f['label'] = y.astype(np.float32)
with open(os.path.join(dirname, 'train.txt'), 'w') as f:
    f.write(train_filename + '\n')
    f.write(train_filename + '\n')
    
# HDF5 is pretty efficient, but can be further compressed.
comp_kwargs = {'compression': 'gzip', 'compression_opts': 1}
with h5py.File(test_filename, 'w') as f:
    f.create_dataset('data', data=Xt, **comp_kwargs)
    f.create_dataset('label', data=yt.astype(np.float32), **comp_kwargs)
with open(os.path.join(dirname, 'test.txt'), 'w') as f:
    f.write(test_filename + '\n')

## logistics regression in Cafee 

Let's define logistic regression in Caffe through Python net specification. This is a quick and natural way to define nets that sidesteps manually editing the protobuf model.

In [0]:
#@title verify drectory {display-mode: "form"}
import os
os.chdir('/content/caffe')
!pwd
# This code will be hidden when the notebook is loaded.


In [0]:
from caffe import layers as L
from caffe import params as P

def logreg(hdf5, batch_size):
    # logistic regression: data, matrix multiplication, and 2-class softmax loss
    n = caffe.NetSpec()
    n.data, n.label = L.HDF5Data(batch_size=batch_size, source=hdf5, ntop=2)
    n.ip1 = L.InnerProduct(n.data, num_output=2, weight_filler=dict(type='xavier'))
    n.accuracy = L.Accuracy(n.ip1, n.label)
    n.loss = L.SoftmaxWithLoss(n.ip1, n.label)
    return n.to_proto()

train_net_path = 'examples/hdf5_classification/logreg_auto_train.prototxt'
with open(train_net_path, 'w') as f:
    f.write(str(logreg('examples/hdf5_classification/data/train.txt', 10)))

test_net_path = 'examples/hdf5_classification/logreg_auto_test.prototxt'
with open(test_net_path, 'w') as f:
    f.write(str(logreg('examples/hdf5_classification/data/test.txt', 10)))

### Define our "solver"

Now, we'll define our "solver" which trains the network by specifying the locations of the train and test nets we defined above, as well as setting values for various parameters used for learning, display, and "snapshotting".

In [0]:
from caffe.proto import caffe_pb2

def solver(train_net_path, test_net_path):
    s = caffe_pb2.SolverParameter()

    # Specify locations of the train and test networks.
    s.train_net = train_net_path
    s.test_net.append(test_net_path)

    s.test_interval = 1000  # Test after every 1000 training iterations.
    s.test_iter.append(250) # Test 250 "batches" each time we test.

    s.max_iter = 1000000      # # of times to update the net (training iterations)

    # Set the initial learning rate for stochastic gradient descent (SGD).
    s.base_lr = 0.01        

    # Set `lr_policy` to define how the learning rate changes during training.
    # Here, we 'step' the learning rate by multiplying it by a factor `gamma`
    # every `stepsize` iterations.
    s.lr_policy = 'step'
    s.gamma = 0.1
    s.stepsize = 5000

    # Set other optimization parameters. Setting a non-zero `momentum` takes a
    # weighted average of the current gradient and previous gradients to make
    # learning more stable. L2 weight decay regularizes learning, to help prevent
    # the model from overfitting.
    s.momentum = 0.9
    s.weight_decay = 5e-4

    # Display the current training loss and accuracy every 1000 iterations.
    s.display = 1000

    # Snapshots are files used to store networks we've trained.  Here, we'll
    # snapshot every 10K iterations -- just once at the end of training.
    # For larger networks that take longer to train, you may want to set
    # snapshot < max_iter to save the network and training state to disk during
    # optimization, preventing disaster in case of machine crashes, etc.
    s.snapshot = 10000
    s.snapshot_prefix = 'examples/hdf5_classification/data/train'

    # We'll train on the CPU for fair benchmarking against scikit-learn.
    # Changing to GPU should result in much faster training!
    s.solver_mode = caffe_pb2.SolverParameter.CPU
    
    return s

solver_path = 'examples/hdf5_classification/logreg_solver.prototxt'
with open(solver_path, 'w') as f:
    f.write(str(solver(train_net_path, test_net_path)))

### Run in python

Time to learn and evaluate our Caffeinated logistic regression in Python.

>cpu is significantly faster in here

In [0]:
%%timeit
caffe.set_mode_cpu()
solver = caffe.get_solver(solver_path)
solver.solve()

accuracy = 0
batch_size = solver.test_nets[0].blobs['data'].num
test_iters = int(len(Xt) / batch_size)
for i in range(test_iters):
    solver.test_nets[0].forward()
    accuracy += solver.test_nets[0].blobs['accuracy'].data
accuracy /= test_iters

print("Accuracy: {:.3f}".format(accuracy))

### Run in command

Do the same through the command line interface for detailed output on the model and solving.

In [0]:
!build/tools/caffe train -solver examples/hdf5_classification/logreg_solver.prototxt

If you look at output or the `logreg_auto_train.prototxt`, you'll see that the model is simple logistic regression.
We can make it a little more advanced by introducing a non-linearity between weights that take the input and weights that give the output -- now we have a two-layer network.
That network is given in `nonlinear_auto_train.prototxt`, and that's the only change made in `nonlinear_logreg_solver.prototxt` which we will now use.

The final accuracy of the new network should be higher than logistic regression!

In [0]:
from caffe import layers as L
from caffe import params as P

def nonlinear_net(hdf5, batch_size):
    # one small nonlinearity, one leap for model kind
    n = caffe.NetSpec()
    n.data, n.label = L.HDF5Data(batch_size=batch_size, source=hdf5, ntop=2)
    # define a hidden layer of dimension 40
    n.ip1 = L.InnerProduct(n.data, num_output=40, weight_filler=dict(type='xavier'))
    # transform the output through the ReLU (rectified linear) non-linearity
    n.relu1 = L.ReLU(n.ip1, in_place=True)#By setting the bottom and the top blob to be the same we can tell Caffe to do "in-place",ReLU,Dropout,BatchNorm,Scale
    # score the (now non-linear) features
    n.ip2 = L.InnerProduct(n.ip1, num_output=2, weight_filler=dict(type='xavier'))
    # same accuracy and loss as before
    n.accuracy = L.Accuracy(n.ip2, n.label)
    n.loss = L.SoftmaxWithLoss(n.ip2, n.label)
    return n.to_proto()

train_net_path = 'examples/hdf5_classification/nonlinear_auto_train.prototxt'
with open(train_net_path, 'w') as f:
    f.write(str(nonlinear_net('examples/hdf5_classification/data/train.txt', 10)))

test_net_path = 'examples/hdf5_classification/nonlinear_auto_test.prototxt'
with open(test_net_path, 'w') as f:
    f.write(str(nonlinear_net('examples/hdf5_classification/data/test.txt', 10)))

solver_path = 'examples/hdf5_classification/nonlinear_logreg_solver.prototxt'
with open(solver_path, 'w') as f:
    f.write(str(solver(train_net_path, test_net_path)))

In [0]:
%%timeit
# caffe.set_device(0)
# caffe.set_mode_gpu()
caffe.set_mode_cpu()
solver = caffe.get_solver(solver_path)
solver.solve()

accuracy = 0
batch_size = solver.test_nets[0].blobs['data'].num
test_iters = int(len(Xt) / batch_size)
for i in range(test_iters):
    solver.test_nets[0].forward()
    accuracy += solver.test_nets[0].blobs['accuracy'].data
accuracy /= test_iters

print("Accuracy: {:.3f}".format(accuracy))

Do the same through the command line interface for detailed output on the model and solving.

In [0]:
!build/tools/caffe train -solver examples/hdf5_classification/nonlinear_logreg_solver.prototxt

In [0]:
# Clean up (comment this out if you want to examine the hdf5_classification/data directory).
shutil.rmtree(dirname)